### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import attention
from collections import deque
import pickle
import pyreader
import numpy as np

In [2]:
data= 'data_samples/'
data_path='data_samples/mapping.map'
pattern = 'preprocess.part*'
hidden_size = 10
seq_length = 1
batch_size = 100
num_samples = 0 # 원래는 100개
with open(data_path, "rb") as f:
        word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
num_samples = 10
attention_num=5
max_attention=3
lambda_type='state'
keep_prob=0.9,
num_layer=1
max_grad_norm=3
learning_rate = 0.01
files = [y for x in os.walk(data) for y in iglob(os.path.join(x[0], pattern))]

In [3]:
current_file = deque(files).popleft()
with open(current_file, 'rb') as f:
    current_data = pickle.load(f)[:100]


In [4]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, attention_num], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = attention.AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=True,
                                 attention_num= attention_num,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    session.run(tf.global_variables_initializer())
    loss = a.loss
    minimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    for i, seq_batch in enumerate(current_data):
        actual_lengths = seq_batch.actual_lengths
        identifier_usage = seq_batch.identifier_usage
        inputs = seq_batch.inputs
        masks = tf.transpose(seq_batch.masks,[0,2,1]).eval()
        targets = seq_batch.targets
    
        feed_dict = {input_data_:inputs, targets_:targets, masks_:masks}
        loss_, _ = session.run([loss, minimize], feed_dict= feed_dict)
        print('loss', loss_[0])

Constructing Attention Cell
Constructing Basic Model
Constructing Attention Model
loss 21.527
loss 18.6176
loss 7.34083
loss 5.84947
loss 6.41164
loss 2.16351
loss 6.47098
loss 5.62905
loss 2.29788
loss 5.89627
loss 1.31611
loss 5.67976
loss 5.82908
loss 2.89021
loss 6.69118
loss 6.83359
loss 0.378443
loss 6.60096
loss 6.41716
loss 3.72244
loss 4.99617
loss 4.4364
loss 4.1011
loss 2.9816
loss 0.695985
loss 0.534056
loss 0.731166
loss 5.68194
loss 4.20967
loss 5.43589
loss 4.41657
loss 5.46643
loss 5.9858
loss 0.707065
loss 5.08052
loss 4.33935
loss 4.7107
loss 3.82091
loss 4.46776
loss 2.53735
loss 1.69741
loss 4.05327
loss 5.20906
loss 4.99942
loss 3.97721
loss 1.20468
loss 3.72869
loss 5.03813
loss 3.99473
loss 4.98367
loss 3.94196
loss 4.61696
loss 4.49635
loss 4.40983
loss 4.2883
loss 4.38781
loss 4.04479
loss 4.31966
loss 4.67642
loss 3.64807
loss 4.47796
loss 4.25339
loss 4.55189
loss 2.07297
loss 2.68212
loss 2.49979
loss 1.86986
loss 2.27378
loss 4.54858
loss 4.84922
loss 1.610